# 向量数据库

## 主要功能

- 高效的相似性搜索：利用专门的索引技术（如 HNSW, IVF），在数十亿级别的向量中实现毫秒级的近似最近邻（ANN）查询，快速找到与给定查询最相似的数据。

- 高维数据存储与管理：专门为存储高维向量（通常维度成百上千）而优化，支持对向量数据进行增、删、改、查等基本操作。

- 丰富的查询能力：除了基本的相似性搜索，还**支持按标量字段过滤查询**（例如，在搜索相似图片的同时，指定年份 > 2023）、范围查询和聚类分析等。

- 可扩展与高可用：采用分布式架构，具备良好的水平扩展能力和容错性，能够通过增加节点来应对数据量的增长。

- 数据与模型生态集成：与主流的 AI 框架（如 LangChain, LlamaIndex）和机器学习工作流无缝集成。

## 向量数据库 vs 传统数据库

向量数据库 (Vector Database) 是一种专门设计用于高效存储、管理和查询高维向量的数据库系统。

与传统数据库的主要差异如下：

| **维度** | **向量数据库** | **传统数据库 (RDBMS)** |
| :--- | :--- | :--- |
| **核心数据类型** | 高维向量 (Embeddings) | 结构化数据 (文本、数字、日期) |
| **查询方式** | **相似性搜索** (ANN) | **精确匹配** |
| **索引机制** | HNSW, IVF, LSH 等 ANN 索引 | B-Tree, Hash Index |
| **主要应用场景** | AI 应用、RAG、推荐系统、图像/语音识别 | 业务系统 (ERP, CRM)、金融交易、数据报表 |
| **数据规模** | 轻松应对千亿级向量 | 通常在千万到亿级行数据，更大规模需复杂分库分表 |
| **性能特点** | 高维数据检索性能极高，计算密集型 | 结构化数据查询快，高维数据查询性能呈指数级下降 |
| **一致性** | 通常为最终一致性 | 强一致性 (ACID 事务) |

在构建现代 AI 应用时，通常会将两者结合使用：

利用传统数据库存储业务元数据和结构化信息，而向量数据库则专门负责处理和检索由 AI 模型产生的海量向量数据。

## 工作原理

向量数据库通常采用四层架构，实现高效相似性搜索：

1. 存储层：存储向量数据和元数据，优化存储效率，支持分布式存储。
   
2. 索引层：维护索引算法（HNSW、LSH、PQ等），创建和优化索引，支持索引调整。
   
3. 查询层：处理查询请求，支持混合查询，实现查询优化。
   
4. 服务层：管理客户端连接，提供监控和日志，实现安全管理。

主要技术手段包括：

- 基于树的方法：如 Annoy 使用的随机投影树，通过树形结构实现对数复杂度的搜索

- 基于哈希的方法：如 LSH（局部敏感哈希），通过哈希函数将相似向量映射到同一“桶”

- 基于图的方法：如 HNSW（分层可导航小世界图），通过多层邻近图结构实现快速搜索

- 基于量化的方法：如 Faiss 的 IVF 和 PQ，通过聚类和量化压缩向量


## 主流向量数据库介绍

Pinecone 是一款完全托管的向量数据库服务，采用Serverless架构设计。适合企业级生产环境、高并发场景和大规模部署。

Milvus 是一款开源的分布式向量数据库，采用分布式架构设计。适合大规模部署、高性能要求的场景，以及需要自定义开发的开源项目。

Qdrant 是一款高性能的开源向量数据库，供多种索引策略和向量混合搜索功能。适合性能敏感应用、高并发场景以及中小规模部署。

## FAISS

FAISS 是一个由 Facebook AI Research 开发的高性能库，专门用于高效的相似性搜索和密集向量聚类。

与 ChromaDB 等数据库不同，FAISS 本质上是一个算法库，它将索引直接保存为本地文件（一个 .faiss 索引文件和一个 .pkl 映射文件），

而非运行一个数据库服务。这种方式轻量且高效。

### 基础示例

配置嵌入模型

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv() 

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model=os.getenv("SiliconFlow_EMBEDDING_MODEL"),
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("SiliconFlow_BASE_URL"),
)


示例文本

In [ ]:
texts = [
    "FAISS是一个用于高效相似性搜索和密集向量聚类的库。",
    "张三是法外狂徒",
    "LangChain是一个用于开发由语言模型驱动的应用程序的框架。",
    "FAISS"
]

docs = [Document(page_content=t) for t in texts]

docs


[Document(metadata={}, page_content='FAISS是一个用于高效相似性搜索和密集向量聚类的库。'),
 Document(metadata={}, page_content='张三是法外狂徒'),
 Document(metadata={}, page_content='LangChain是一个用于开发由语言模型驱动的应用程序的框架。'),
 Document(metadata={}, page_content='FAISS')]

创建向量存储并保存到本地

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(docs, embeddings)
local_faiss_path = "./faiss_index_store"
vectorstore.save_local(local_faiss_path)


加载索引并执行查询，加载时需指定相同的嵌入模型，并允许反序列化

In [ ]:
loaded_vectorstore = FAISS.load_local(
    local_faiss_path,
    embeddings,
    allow_dangerous_deserialization=True
)


相似性搜索

In [ ]:
query = "FAISS是做什么的？"
results = loaded_vectorstore.similarity_search(query, k=3)

print(f"\n查询: '{query}'")
print("相似度文档排序:")
for doc in results:
    print(f"- {doc.page_content}")



查询: 'FAISS是做什么的？'
相似度文档排序:
- FAISS
- 张三是法外狂徒
- FAISS是一个用于高效相似性搜索和密集向量聚类的库。


不知道为什么，"张三是法外狂徒"的相似度比"FAISS是一个用于高效相似性搜索和密集向量聚类的库"高？？？？？？